# 머신러닝 모델 개발 워크플로우 

## 1. 모듈 및 데이터 로딩
- **설명**: 필요한 라이브러리 임포트 및 데이터셋 불러오기
- **가이드라인**:
  - 기본 라이브러리: `pandas`, `numpy`, `scikit-learn`, `matplotlib`, `seaborn`
  - 데이터 크기가 큰 경우: `dask` 또는 `polars` 사용
  - 이미지 데이터: `opencv`, `PIL` 추가
  - 시계열 데이터: `statsmodels`, `pmdarima` 추가

---

## 2. 데이터 전처리
- **설명**: 결측치, 이상치, 중복값 처리 및 데이터 타입 변환
- **결측치 처리 가이드라인**:
  - **결측치가 5% 이하**: 평균(연속형) 또는 최빈값(범주형)으로 대체
  - **결측치가 5~30%**: KNN Imputer 또는 보간법(interpolation) 사용
  - **결측치가 30% 이상**: 해당 피쳐 제거 검토
  - **시계열 데이터**: 선형보간, 포워드필(forward fill) 또는 백워드필(backward fill) 사용
  - **타겟 변수의 결측치**: 해당 행 제거 필수

- **이상치 처리 가이드라인**:
  - **정규분포 데이터**: Z-score 방법 (|Z| > 3 제거)
  - **편포된 데이터**: IQR 방법 (Q1 - 1.5×IQR, Q3 + 1.5×IQR 범위 초과 제거)
  - **비즈니스 의존적**: 도메인 전문가와 상담 후 판단

- **중복값 처리**: `drop_duplicates()` 사용, 전체 행이 같은 경우만 제거

- **데이터 타입 변환**: 범주형은 `category` 타입으로, 날짜는 `datetime64` 타입으로 변환

---

## 3. 데이터 탐색 및 분석(EDA)
- **설명**: 데이터의 분포, 통계, 상관관계 파악 및 시각화
- **가이드라인**:
  - 기술통계: `describe()`, `info()`, `value_counts()` 확인
  - 분포 시각화: 히스토그램, 박스플롯, 바이올린플롯
  - 상관관계: 히트맵으로 피어슨/스피어만 상관계수 확인
  - 타겟 변수 불균형 확인: 클래스 불균형 데이터는 **전략 수립 필요** (SMOTE, 클래스 가중치 조정)
  - 이상 패턴 감지: 특정 값의 과도한 집중 여부 확인

---

## 4. 인코딩
- **설명**: 범주형 변수를 수치형으로 변환
- **인코더 선택 가이드라인**:

| 상황 | 추천 인코더 | 이유 |
|------|-----------|------|
| **선형 모델** (로지스틱 회귀, 선형회귀) | **One-Hot Encoding** | 카테고리 간의 인위적 순서 관계 제거 필수 |
| **트리 기반 모델** (결정트리, 랜덤포레스트) | **Label Encoding** 또는 인코딩 불필요 | 트리는 순서 관계를 고려하지 않음 |
| **카테고리 수 < 10개** | **One-Hot Encoding** | k-1 개의 더미 변수 생성 (다중공선성 방지) |
| **카테고리 수 > 100개** (고카디널리티) | **Target Encoding** | 카테고리 수가 너무 많으면 One-Hot이 계산 비효율적 |
| **순서 있는 범주형** (낮음, 중간, 높음) | **Ordinal Encoding** | 순서 정보 보존 필요 |
| **신경망 모델** | **One-Hot Encoding** | 입력층에서 각 카테고리를 별도 노드로 처리 |
| **타겟 변수** | **Label Encoding** | 타겟은 각 클래스에 정수 할당 |

---

## 5. 피쳐 엔지니어링
- **설명**: 새로운 피쳐 생성, 피쳐 선택 및 변환
- **가이드라인**:
  - **새로운 피쳐 생성**: 도메인 지식 활용 (예: 나이에서 연령대 범주 생성)
  - **다항 피쳐**: 낮은 차수 다항식 (2~3차) 추가 (과적합 위험)
  - **상호작용 피쳐**: 중요한 피쳐 간의 곱셈/나눗셈 조합
  - **피쳐 선택 방법**:
    - **필터 방법**: 상관계수, chi-square, 정보 획득
    - **래퍼 방법**: 순환 특성 제거 (RFE), 전진/후진 선택
    - **임베딩 방법**: L1 정규화 (Lasso), 트리 기반 피쳐 중요도

---

## 6. 피쳐 및 타겟 분리
- **설명**: X (독립변수)와 y (종속변수) 분리
- **가이드라인**:
  - 타겟 변수는 반드시 y에 할당
  - 피쳐는 X에 할당 (타겟 제외, ID 열 제외)
  - 시계열 데이터의 경우 시간 순서 보존 중요

---

## 7. 데이터셋 분할
- **설명**: 학습 데이터와 테스트 데이터 분리
- **가이드라인**:
  - **기본 분할**: 70/30 또는 80/20 비율
  - **불균형 데이터**: `stratify=y` 옵션으로 클래스 비율 유지
  - **시계열 데이터**: 시간 순서대로 분할 (무작위 분할 금지)
  - **소규모 데이터 (< 1000개)**: 더 큰 비율로 학습 데이터 할당 (90/10)

---

## 8. 피쳐 스케일링
- **설명**: 피쳐를 동일한 스케일로 정규화 또는 표준화
- **스케일러 선택 가이드라인**:

| 스케일러 | 공식 | 사용 상황 | 장점 | 단점 |
|---------|------|---------|------|------|
| **StandardScaler** | (X - mean) / std | **정규분포 데이터**, KNN, SVM, 신경망 | 이상치 영향 적음, 표준정규분포 생성 | 극단값에 영향받음 |
| **MinMaxScaler** | (X - min) / (max - min) | **경계 범위 [0,1] 필요**, 이미지 픽셀값, 신경망 | 직관적, 범위 제한 | **이상치에 매우 민감** |
| **RobustScaler** | (X - median) / IQR | **이상치 많은 데이터**, 부분적 결손 데이터 | 이상치 강건함, 중앙값 기반 | 정규분포 보장 안함 |
| **MaxAbsScaler** | X / max(\|X\|) | **희소 데이터** (sparse data) | 희소성 유지 | 극단값에 영향받음 |

- **결정 플로우**:
  1. **이상치가 있는가?** → 예: RobustScaler / 아니오: StandardScaler
  2. **이미지 또는 [0,1] 범위 필요?** → 예: MinMaxScaler
  3. **희소 데이터?** → 예: MaxAbsScaler

---

## 9. 머신러닝 모델 선정 및 학습
- **설명**: 문제 유형에 맞는 모델 선택 및 초기 학습
- **모델 선택 가이드라인**:

### 분류(Classification) 문제

| 상황 | 추천 모델 | 이유 | 주의사항 |
|------|---------|------|---------|
| **작은 데이터셋** (< 10K) | **KNN**, **로지스틱 회귀** | 간단, 과적합 위험 낮음 | KNN은 느린 예측 속도 |
| **중간 크기 데이터** | **결정트리**, **랜덤포레스트** | 빠른 학습, 해석 가능 | 트리는 과적합 경향 |
| **높은 성능 필요** | **Gradient Boosting** (XGBoost, LightGBM) | 높은 정확도 | 하이퍼파라미터 민감, 느린 학습 |
| **불균형 클래스** | **로지스틱 회귀** + 클래스 가중치 또는 **SMOTE** | 소수 클래스 고려 | - |
| **선형 분리 가능** | **로지스틱 회귀**, **SVM** (선형) | 해석 가능, 빠름 | 비선형 경계 표현 불가 |
| **비선형 경계** | **SVM** (RBF), **신경망** | 복잡한 패턴 학습 | 하이퍼파라미터 많음 |
| **첫 시작 모델** | **로지스틱 회귀** | 베이스라인 설정 | 성능 개선 여지 많음 |

### 회귀(Regression) 문제

| 상황 | 추천 모델 | 이유 |
|------|---------|------|
| **선형 관계** | **선형 회귀** | 간단, 빠름, 해석 가능 |
| **비선형 관계** | **결정트리**, **랜덤포레스트** | 복잡한 패턴 학습 |
| **높은 정확도** | **Gradient Boosting** | 최고 성능 |
| **작은 데이터** | **서포트 벡터 회귀(SVR)** | 일반화 능력 우수 |

---

## 10. 교차 검증(Cross-Validation)
- **설명**: K-Fold 등으로 모델 성능 검증 및 과적합 방지
- **가이드라인**:
  - **기본 K-Fold**: K=5 또는 K=10 (데이터 크기에 따라)
  - **소규모 데이터 (< 100개)**: Leave-One-Out CV (LOOCV)
  - **불균형 데이터**: **StratifiedKFold** 사용 (클래스 비율 유지)
  - **시계열 데이터**: **TimeSeriesSplit** 사용 (시간 순서 유지)
  - **평가 지표**: 평균 점수와 표준편차 모두 확인 (분산 측정)

---

## 11. 하이퍼파라미터 튜닝
- **설명**: Grid Search 또는 Random Search로 최적 파라미터 선택
- **하이퍼파라미터 튜닝 가이드라인**:

### Grid Search vs Random Search

| 방법 | Grid Search | Random Search |
|------|-----------|----------------|
| **검색 방식** | 모든 조합 탐색 | 임의 조합 샘플링 |
| **계산 비용** | 높음 | 낮음 |
| **최적해 발견** | 확정적 | 확률적 |
| **사용 시기** | 파라미터 개수 ≤ 3개 | 파라미터 개수 > 3개 |
| **하이퍼파라미터 민감도 높음** | 권장 | 덜 권장 |

### 모델별 주요 하이퍼파라미터 튜닝

**KNN**
- **n_neighbors (K)**: 1~20 범위에서 교차검증으로 선택
  - K 작음: 과적합 위험↑
  - K 큼: 과소적합 위험↑
- **distance metric**: 'euclidean', 'manhattan' (데이터 특성에 따라)
- **weights**: 'uniform' 또는 'distance'

**결정트리**
- **max_depth**: 3~20 범위 (너무 깊으면 과적합)
- **min_samples_split**: 2~10
- **min_samples_leaf**: 1~5

**랜덤포레스트**
- **n_estimators**: 50~500 (많을수록 좋지만 계산 비용↑)
- **max_depth**: 제한 (기본값은 제한 없음, 과적합 위험)
- **min_samples_leaf**: 1~5

**Gradient Boosting**
- **learning_rate**: 0.01~0.1 (낮을수록 안정적, 느림)
- **n_estimators**: 100~1000
- **max_depth**: 3~7 (깊을수록 과적합 위험↑)
- **subsample**: 0.5~1.0 (데이터 샘플링 비율)

---

## 12. 최종 모델 생성
- **설명**: 최적 파라미터로 전체 학습 데이터에서 재학습
- **가이드라인**:
  - 튜닝된 하이퍼파라미터로 모델 재생성
  - 전체 학습 데이터 (train + validation) 사용하여 학습
  - 모델 저장: `joblib.dump()` 또는 `pickle`

---

## 13. 테스트 데이터 성능 평가
- **설명**: 평가 메트릭으로 모델 성능 측정
- **분류 문제 평가 지표**:
  - **정확도 (Accuracy)**: 전체 데이터 중 올바른 예측 비율
  - **정밀도 (Precision)**: 양성 예측 중 실제 양성 비율 (위양성 최소화)
  - **재현율 (Recall)**: 실제 양성 중 올바른 예측 비율 (위음성 최소화)
  - **F1-Score**: Precision과 Recall의 조화평균
  - **ROC-AUC**: 분류 임계값 변화에 따른 성능 평가

- **회귀 문제 평가 지표**:
  - **MSE (Mean Squared Error)**: 평균 제곱 오차
  - **MAE (Mean Absolute Error)**: 평균 절대 오차
  - **R² (결정계수)**: 모델이 설명하는 분산 비율

- **평가 지표 선택 가이드라인**:
  - **불균형 데이터**: Precision, Recall, F1-Score 우선
  - **균형 잡힌 데이터**: Accuracy 신뢰성↑
  - **비즈니스 중심**: 위양성/위음성 비용에 따라 선택

---

## 14. 데이터 예측
- **설명**: 새로운 데이터에 대한 예측 수행
- **가이드라인**:
  - **전처리 일관성 필수**: 학습 데이터와 동일한 전처리 적용
  - **스케일러 재사용**: 학습 데이터로 fit한 스케일러로 transform
  - **인코더 재사용**: 학습 데이터로 fit한 인코더 적용
  - **확률 예측**: `predict_proba()` 사용 (임계값 조정 가능)
  - **신뢰도 점수**: 모델의 예측 신뢰도 함께 제공

---

## 최종 워크플로우 요약

```
1. 모듈 및 데이터 로딩 → 
2. 데이터 전처리 → 
3. EDA → 
4. 인코딩 (범주형 변수) → 
5. 피쳐 엔지니어링 → 
6. 피쳐/타겟 분리 → 
7. 데이터셋 분할 → 
8. 피쳐 스케일링 → 
9. 모델 선정 및 학습 → 
10. 교차 검증 → 
11. 하이퍼파라미터 튜닝 → 
12. 최종 모델 생성 → 
13. 테스트 성능 평가 → 
14. 예측
```

데이터의 특성과 문제 유형에 따라 일부 단계는 반복하거나 순서를 조정